In [1]:
import pandas as pd
test_df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/建阳站多模型验证期结果.csv",
                      parse_dates=["Date"])
test_df_copy = test_df
test_df_copy = test_df_copy.iloc[:, :2]
test_df.reset_index(drop=True)
test_df = test_df.set_index('Date')
"""
读取训练数据
"""
train_df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/建阳站多模型率定期结果.csv",
                       parse_dates=["Date"])
train_df_copy = train_df
train_df_copy = train_df_copy.iloc[:, :2]
train_df.reset_index(drop=True)
train_df = train_df.set_index('Date')
train_df

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,LSTM,RNN
Date,,,,,,,,,
2000-03-19,146.646083,113.312715,52.709690,180.82450,161.918257,225.542370,151.506776,210.58240,203.54922
2000-03-20,126.275542,77.366574,45.722771,132.35930,151.888716,122.160250,149.861280,140.79071,100.39914
2000-03-21,116.957458,67.217913,43.718240,109.94270,120.206044,92.493935,132.048647,131.23845,104.74185
2000-03-22,107.373583,69.118466,43.801287,97.71193,107.209378,101.336530,131.638650,116.44574,101.35420
2000-03-23,157.025792,112.531645,62.675173,156.76230,128.057335,209.905500,122.400853,198.44711,141.74525
...,...,...,...,...,...,...,...,...,...
2013-03-22,179.575000,202.368997,133.070349,224.24750,301.949810,167.668520,214.686714,177.10118,144.76294
2013-03-23,254.883333,188.156255,102.059662,170.45160,188.006705,113.537680,152.147893,139.23248,154.90280
2013-03-24,518.239583,253.698090,126.938355,199.69960,207.666900,218.423450,224.147323,215.14694,187.25436


In [2]:
# 导入进行线性回归的包

from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np
from tool import Indicator
"""
案例一：XAJ+TANK
"""
# 处理数据
trainX_xaj = train_df.iloc[:, 1]
trainX_tank = train_df.iloc[:, 3]
trainy1 = train_df.iloc[:, :1]
trainx1 = pd.merge(trainX_xaj, trainX_tank, on="Date", how="left")
train_data1=pd.merge(trainy1,trainx1,on="Date",how="left")
testX_xaj = test_df.iloc[:, 1]
testX_tank = test_df.iloc[:, 3]
testy1 = test_df.iloc[:, :1]
testx1 = pd.merge(testX_xaj, testX_tank, on="Date", how="left")
test_data1=pd.merge(testy1,testx1,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled1 = scaler.fit_transform(train_data1)
df_for_testing_scaled1 = scaler.transform(test_data1)

# 划分数据
trainX1 = df_for_training_scaled1[:, 1:]
trainY1 = df_for_training_scaled1[:, :1]
testX1 = df_for_testing_scaled1[:, 1:]
testY1 = df_for_testing_scaled1[:, :1]


# XGBoost
parameters = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.15, 0.025],
    'n_estimators': [50, 100, 200],
    'subsample':[ 0.7, 0.8, 0.9, 1]
}
xlf = xgb.XGBRegressor()
grid_search = GridSearchCV(xlf, param_grid=parameters, cv=5)
grid_search = grid_search.fit(trainX1, trainY1)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict1 = model1.predict(trainX1)
train_predict1 = pd.DataFrame(train_predict1)
train_predict1.columns = ["RF1"]

test_predict1 = model1.predict(testX1)
test_predict1 = pd.DataFrame(test_predict1)
test_predict1.columns = ["RF1"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict1,len(test_data1.columns), axis=-1)
test_pred1 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict1),len(test_data1.columns))))[:, 0]
test_pred_copy1 = test_pred1
test_pred1 = pd.DataFrame(test_pred1)
test_pred1.columns = ["XGBoost1"]

prediction_copies_array = np.repeat(train_predict1,len(test_data1.columns), axis=-1)
train_pred1 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict1),len(test_data1.columns))))[:, 0]
train_pred_copy1 = train_pred1
train_pred1=pd.DataFrame(train_pred1)
train_pred1.columns=['XGBoost1']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred1['XGBoost1'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred1['XGBoost1'], test_df_copy['Qobs'])

最佳参数组合：{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.9}
*******************率定期*******************
MAE:42.73587338478338
RMSE:93.52885662485674
NSE:0.8710184556649678
RE:0.4108867890664687
Qmaxe:0.04376465604511071
Qmine:3.434900906929874
*******************验证期*******************
MAE:57.72064639423327
RMSE:141.67421753104497
NSE:0.7502022157694171
RE:-8.062691639604667
Qmaxe:0.23647912400305168
Qmine:0.9654709808959321


(57.72064639423327,
 141.67421753104497,
 0.7502022157694171,
 -8.062691639604667,
 0.23647912400305168,
 0.9654709808959321)

In [3]:
"""
案例二：BP+RNN
"""
# 处理数据
trainX_BP = train_df.iloc[:, 5]
trainX_RNN = train_df.iloc[:, 7]
trainy2 = train_df.iloc[:, :1]
trainx2 = pd.merge(trainX_BP, trainX_RNN, on="Date", how="left")
testX_BP = test_df.iloc[:, 5]
testX_RNN = test_df.iloc[:, 7]
testy2 = test_df.iloc[:, :1]
testx2 = pd.merge(testX_BP, testX_RNN, on="Date", how="left")
train_data2=pd.merge(trainy2,trainx2,on="Date",how="left")
test_data2=pd.merge(testy2,testx2,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled2 = scaler.fit_transform(train_data2)
df_for_testing_scaled2 = scaler.transform(test_data2)

# 划分数据
trainX2 = df_for_training_scaled2[:, 1:]
trainY2 = df_for_training_scaled2[:, :1]
testX2 = df_for_testing_scaled2[:, 1:]
testY2 = df_for_testing_scaled2[:, :1]


# 随机森林回归器
parameters = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.15, 0.025],
    'n_estimators': [50, 100, 200],
    'subsample':[ 0.7, 0.8, 0.9, 1]
}
xlf = xgb.XGBRegressor()
grid_search = GridSearchCV(xlf, param_grid=parameters, cv=5)
grid_search = grid_search.fit(trainX2, trainY2)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict2 = model1.predict(trainX2)
train_predict2 = pd.DataFrame(train_predict2)
train_predict2.columns = ["XGBoost2"]

test_predict2 = model1.predict(testX2)
test_predict2 = pd.DataFrame(test_predict2)
test_predict2.columns = ["XGBoost2"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict2,len(test_data2.columns), axis=-1)
test_pred2 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict2),len(test_data2.columns))))[:, 0]
test_pred_copy2 = test_pred2
test_pred2 = pd.DataFrame(test_pred2)
test_pred2.columns = ["XGBoost2"]

prediction_copies_array = np.repeat(train_predict2,len(test_data2.columns), axis=-1)
train_pred2 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict2),len(test_data2.columns))))[:, 0]
train_pred_copy2 = train_pred2
train_pred2=pd.DataFrame(train_pred2)
train_pred2.columns=['XGBoost2']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred2['XGBoost2'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred2['XGBoost2'], test_df_copy['Qobs'])

最佳参数组合：{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1}
*******************率定期*******************
MAE:36.47000211443946
RMSE:89.70903801449288
NSE:0.8813388039930139
RE:0.4568810023353179
Qmaxe:0.030088491960179042
Qmine:2.1148497183339776
*******************验证期*******************
MAE:52.51972212042175
RMSE:143.08302989226837
NSE:0.745209522608854
RE:-5.075403555516015
Qmaxe:0.3455799335317012
Qmine:0.6159285456098844


(52.51972212042175,
 143.08302989226837,
 0.745209522608854,
 -5.075403555516015,
 0.3455799335317012,
 0.6159285456098844)

In [4]:
"""
案例三：BP+RNN+XAJ+TANK
"""
# 处理数据
# 处理数据
trainy3 = train_df.iloc[:, :1]
trainx3 = pd.merge(trainx1, trainx2, on="Date", how="left")
testy3 = test_df.iloc[:, :1]
testx3 = pd.merge(testx1, testx2, on="Date", how="left")
train_data3=pd.merge(trainy3,trainx3,on="Date",how="left")
test_data3=pd.merge(testy3,testx3,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled3 = scaler.fit_transform(train_data3)
df_for_testing_scaled3 = scaler.transform(test_data3)

# 划分数据
trainX3 = df_for_training_scaled3[:, 1:]
trainY3 = df_for_training_scaled3[:, :1]
testX3 = df_for_testing_scaled3[:, 1:]
testY3 = df_for_testing_scaled3[:, :1]


parameters = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.15, 0.025],
    'n_estimators': [50, 100, 200],
    'subsample':[ 0.7, 0.8, 0.9, 1]
}
xlf = xgb.XGBRegressor()
grid_search = GridSearchCV(xlf, param_grid=parameters, cv=5)
grid_search = grid_search.fit(trainX3, trainY3)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict3 = model1.predict(trainX3)
train_predict3 = pd.DataFrame(train_predict3)
train_predict3.columns = ["XGBoost3"]

test_predict3 = model1.predict(testX3)
test_predict3 = pd.DataFrame(test_predict3)
test_predict3.columns = ["XGBoost3"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict3,len(test_data3.columns), axis=-1)
test_pred3= scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict3),len(test_data3.columns))))[:, 0]
test_pred_copy3 = test_pred3
test_pred3 = pd.DataFrame(test_pred3)
test_pred3.columns = ["XGBoost3"]

prediction_copies_array = np.repeat(train_predict3,len(test_data3.columns), axis=-1)
train_pred3 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict3),len(test_data3.columns))))[:, 0]
train_pred_copy3 = train_pred3
train_pred3=pd.DataFrame(train_pred3)
train_pred3.columns=['XGBoost3']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred3['XGBoost3'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred3['XGBoost3'], test_df_copy['Qobs'])

最佳参数组合：{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1}
*******************率定期*******************
MAE:35.113922032552615
RMSE:84.27226380540753
NSE:0.8952857845448523
RE:0.4582018737355483
Qmaxe:0.034179205609345786
Qmine:2.37471930189816
*******************验证期*******************
MAE:51.98776274853689
RMSE:140.66234207747144
NSE:0.7537577193100944
RE:-5.114857624310182
Qmaxe:0.32174781469973057
Qmine:0.7076961224512849


(51.98776274853689,
 140.66234207747144,
 0.7537577193100944,
 -5.114857624310182,
 0.32174781469973057,
 0.7076961224512849)

In [6]:
test_df_full = pd.concat([test_df_copy, test_pred1, test_pred2, test_pred3], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/JY_XGBoost1Test.csv")
train_df_full = pd.concat([train_df_copy, train_pred1, train_pred2, train_pred3], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/JY_XGBoost1Train.csv")
test_df_full

,Qobs,XGBoost1,XGBoost2,XGBoost3
Date,,,,
2013-03-27,357.285417,46.729073,555.195129,530.587280
2013-03-28,266.375000,46.729073,248.987610,197.105453
2013-03-29,292.975000,46.729073,248.987610,197.105453
2013-03-30,279.295833,83.401169,331.893494,281.143646
2013-03-31,263.750000,80.001541,242.123825,183.926468
...,...,...,...,...
2018-10-26,39.116250,62.064590,51.340519,53.530777
2018-10-27,46.711250,99.087975,102.818413,99.458435
2018-10-28,70.970417,89.584068,102.818413,94.906242
